# Leoilab - Deep learning workshop - Techfestival 2018


This workshop will take you through training of a deep neural network using Keras. 

We will train a network to diagnose malignant vs benign melanoma.

We'll cover the following topics:
* Loading data
* Training a basic network
* Transfer learning
* Adding data augmentation

## Basics of Jupyter Notebooks



## Loading data

In [ ]:
def plot_training_epochs(history):
    plt.figure(figsize=(15, 10))
    plt.plot(history.epoch, history.history['acc'])
    plt.plot(history.epoch, history.history['val_acc'])
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    l = plt.legend(['training accuracy', 'validation accuracy'])

In [ ]:
cats_and_dogs_data = '/home/ubuntu/store/dogscats/1ksample'
melanoma_data = '/home/ubuntu/store/isic-full/data/'

path_to_data = cats_and_dogs_data

In [ ]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img
from keras.applications.imagenet_utils import preprocess_input

import matplotlib.pyplot as plt
import numpy as np
% matplotlib inline

batch_size = 16
image_size = (224, 224)
n_classes = 2

# Create data generators
data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)
train_gen = data_generator.flow_from_directory(path_to_data + '/train', batch_size=batch_size, target_size=image_size)
valid_gen = data_generator.flow_from_directory(path_to_data + '/valid', batch_size=batch_size, target_size=image_size)

The generators `train_gen` and `test_gen` will iterate over training and testing images. 

Each output will be a tuple of image-tensor and label-vector.

Try plotting the first image:

In [ ]:
images, labels = next(train_gen)

In [ ]:
## TODO: try examining the contents of `images` and `labels`

In [ ]:
## TODO: use plt.imshow to plot the images in `images`
##       To convert an array back to an image that can be plotted use `array_to_img(image)`

We see that the image looks a bit odd. This is because we normalize the three channels of the image to decrease training time (done with the `preprocessing_function` provided above).

## Train a basic network

Keras has multiple predefined networks that work well for different tasks under `keras.applications`.

They are normally designed to perform on the 1000-class [ImageNet](https://www.image-net.org/) dataset.

In our case we only have benign / malignant, so we need to specify that we want the networks with two instead of a thousand outputs.

In [ ]:
from keras.applications.resnet50 import ResNet50
from keras.optimizers import Adam

# Create the resnet model
model = ResNet50(classes=2, weights=None)

# Compile model - don't worry about this :)
model.compile('adam', loss='binary_crossentropy', metrics=['accuracy'])

The model we loaded here has `weights=None`, with this we will randomly initialize the weights of the model.

Keras models have methods for doing a bunch of the basic things we might want to do. 

Try playing around with the following:

* `model.summary`: This will give you a (long) description of the network's layers.

* `model.layers`: This will give you a python list of layers, that you can play around with.

* `model.predict`: This will apply the model, and output a probability for each class, for each image. Try applying this to the images you got from the generator above.

* `model.fit_generator`: This will fit the model to the data

In [ ]:
## TODO: run predict on the images from above. You can use np.argmax to get the index with the highest probability.

In [ ]:
## TODO: Run one epoch of training with model.fit_generator.
##       You can also supply the argument `validation_data=...` to run validation on a different generator
##       Try increasing the number of training cycles (epochs) to 5
## TODO: .fit_generator returns a `history`-object.
##       Try plotting the train/valid accuracies with `plot_training_epochs(history)`

In [ ]:
# Tip: If you didn't store the history object from running .fit_generator you can get the output from the previous
#      cell by typing `_`. The following line will store that output to the history variable. Like this:
history = _

## Transfer learning

In transfer learning we take a model that was initially trained on one task and use it for another task.

All models in `keras.applications` are pre-trained on the 1000 classes in ImageNet, with _a lot_ of data for each class. This means that they can learn the low level representations of objects, and we just need to 

In [ ]:
from keras.layers import Dense
from keras.models import Model


# Load the ResNet model - now with pretrained imagenet weights
model_imagenet = ResNet50(weights='imagenet')

# Create an output layer with two classes instead of 1000. 
# The input to that layer should be the second to last layer of the imagenet model.
output_layer = Dense(n_classes, activation='softmax')(model_imagenet.layers[-2].output)

# Now we can build a new model with our self-defined activation output:
model = Model(inputs=model_imagenet.input, outputs=output_layer)

This new model will have all layers trainable. We only want to train the layer we added to the top of the model.

You can set a layer to be trainable like this:

In [ ]:
output_layer.trainable = True

If set to `False` it will be untrainable.

Set all layers in model to not train, except the output layer:

In [ ]:
## TODO: Loop over layers in model.layers and set to not train.

Now we can compile the model.

In [ ]:
# Compile model - don't worry about this :)
model.compile('adam', loss='binary_crossentropy', metrics=['accuracy'])

Now try fitting the generator as above.

In [ ]:
## TODO: run mode.fit_generator on the data as you did in the first section

In [ ]:
history = _

In [ ]:
plot_training_epochs(history)

## Data augmentation

We see a much better training accuracy then before, but we can also see a bit of overfitting. This is because - when training on the same data multiple times - the model will learn to just remember the data is was shown previously.

Since images are very dense with information it is quite easy for this big of a model to remember patterns in each of the images in our data set. To force the model to generalize we can use data augmentation.

Data augmentation adds a small random transformation to an image everytime it's loaded (once per epoch).

The augmentations we use here are flips, rotations, and zoom.

This sure that the model never sees the __exact__ same image twice.

Data augmentation is very easy to add in the image data generator in keras:

In [ ]:
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=40.0, # degrees of rotation
    zoom_range=0.3
)

train_gen = data_generator.flow_from_directory(path_to_data + '/train', batch_size=batch_size, target_size=image_size)
valid_gen = data_generator.flow_from_directory(path_to_data + '/valid', batch_size=batch_size, target_size=image_size)

Now let's have a look at how the augmented images look:

In [ ]:
imgs, labs = next(train_gen)
plt.imshow(array_to_img(imgs[0, ...]))

Now try training the model again. You can let it run for a while. See how many epochs you need to run before getting to overfitting.

In [ ]:
## TODO: use .fit_generator to train the model again

If you want to try increasing the amount of training data, remove `1ksample` from the end of `path_to_data`.